# Lesson 14: 빅데이터와 시각화 – 크롤링 + 시각화

최규빈  
2023-07-26

<a href="https://colab.research.google.com/github/guebin/PP2024WIN/blob/main/posts/Day3/ls14.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

In [1]:
# !pip install yfinance
import numpy as np 
import pandas as pd 
import yfinance as yf

# `yfinance`를 이용한 주식자료 시각화

## A. 크롤링 + 데이터정리

`-` yahoo finance: https://finance.yahoo.com/

**Apple: `'AAPL'`**

<img
src="ls14_files/figure-ipynb/8f6bf0b8-35e6-4324-ae0a-e2628e7bb60b-1-10wk-1_files/figure-ipynb/c2016753-3330-437a-bd4e-d9ef1cd24afb-1-825f3296-8485-439d-8432-e52a19c50910.png"
alt="Apple 코드" />

Apple 코드

**삼성전자: `'005930.KS'`**

<img
src="ls14_files/figure-ipynb/8f6bf0b8-35e6-4324-ae0a-e2628e7bb60b-2-10wk-1_files/figure-ipynb/dd41452f-2b17-47e5-8b5d-ea2c46bba973-1-c2ae28ce-13f7-44e7-8c1f-32055d0535f1.png"
alt="삼성전자 코드" />

삼성전자 코드

`-` 크롤링을 위한 코드

In [2]:
symbols = ['AMZN','AAPL','GOOG','MSFT','NFLX','NVDA','TSLA']
start = '2020-01-01'
end = '2023-11-06'
df = yf.download(symbols,start,end)
df

[                       0%%                      ][**************        29%%                      ]  2 of 7 completed[**************        29%%                      ]  2 of 7 completed[**********************57%%*                     ]  4 of 7 completed[**********************57%%*                     ]  4 of 7 completed[**********************86%%***************       ]  6 of 7 completed[*********************100%%**********************]  7 of 7 completed

`-` 데이터를 살펴보자.

In [3]:
df.stack()

## B. 시각화

`-` tidydata를 생성

In [4]:
df.loc[:,'Adj Close'].stack().reset_index()\
.set_axis(['Date','Company','Price'],axis=1)

`-` 바로 이어서 시각화! – `x='Date', y='Price', color='Company'`

In [5]:
df.loc[:,'Adj Close'].stack().reset_index()\
.set_axis(['Date','Company','Price'],axis=1)\
.plot.line(x='Date',y='Price',color='Company',backend='plotly')

/home/cgb2/anaconda3/envs/r/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result


# 출산율 시각화

## A. 크롤링 + 데이터정리

`-` 대한민국의 저출산문제

ref: <https://ko.wikipedia.org/wiki/대한민국의_저출산>

`-` 위의 url에서 5번째 테이블을 읽고싶다.

-   5번째 테이블: 시도별 출생아 수

In [6]:
df_lst = pd.read_html('https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%A0%80%EC%B6%9C%EC%82%B0')
df = df_lst[4]
df

`-` `'-'`로 표현된 값을 0으로 처리하는게 좋겠음

In [7]:
df.rename({'지역/연도[6]':'지역'},axis=1)\
.set_index(['지역'])\
.applymap(lambda x: 0 if '-' in str(x) else float(x)).reset_index()

/tmp/ipykernel_1311829/1737303492.py:3: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


## B. 시각화1: 전국 출생아수 시각화

`-` 전국으로 집계가 되어있던 정보를 남겨도 좋지만, `groupy+agg` 조합으로
새롭게 계산해보자.

In [8]:
df.rename({'지역/연도[6]':'지역'},axis=1)\
.set_index(['지역'])\
.applymap(lambda x: 0 if '-' in str(x) else float(x)).drop('전국')\
.stack().reset_index().rename({'level_1':'년도',0:'출생아수'},axis=1)\
.groupby('년도').agg({'출생아수':'sum'}).reset_index()\
.plot.line(x='년도',y='출생아수',backend='plotly')

/tmp/ipykernel_1311829/1805618332.py:3: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


## C. 시각화2: 시도별 출생아수 시각화 (line)

In [9]:
df.rename({'지역/연도[6]':'지역'},axis=1)\
.set_index(['지역'])\
.applymap(lambda x: 0 if '-' in str(x) else float(x)).drop('전국')\
.stack().reset_index().rename({'level_1':'년도',0:'출생아수'},axis=1)\
.plot.line(x='년도',y='출생아수',color='지역',backend='plotly')

/tmp/ipykernel_1311829/277281482.py:3: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


## D. 시각화3: 시도별 출생아수 시각화 (area)

`-` 시각화1,시각화2의 정보가 적절히 혼합되어있는 시각화는 없을까?

In [10]:
df.rename({'지역/연도[6]':'지역'},axis=1)\
.set_index(['지역'])\
.applymap(lambda x: 0 if '-' in str(x) else float(x)).drop('전국')\
.stack().reset_index().rename({'level_1':'년도',0:'출생아수'},axis=1)\
.plot.area(x='년도',y='출생아수',color='지역',backend='plotly')

/tmp/ipykernel_1311829/70633050.py:3: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


-   기묘하다..

`-` 해석: 2014년 경기, 2016년 서울, 2019년 경기에 애를 낳지 않는다.

`-` 데이터와 비교

In [11]:
df

## E. 시각화1,2,3 수정

`-` 시각화1의 수정

In [12]:
df.rename({'지역/연도[6]':'지역'},axis=1)\
.set_index(['지역'])\
.applymap(lambda x: 0 if '-' in str(x) else float(x)).drop('전국')\
.applymap(lambda x: x*1000 if x<1000 else x)\
.stack().reset_index().rename({'level_1':'년도',0:'출생아수'},axis=1)\
.groupby('년도').agg({'출생아수':'sum'}).reset_index()\
.plot.line(x='년도',y='출생아수',backend='plotly')

/tmp/ipykernel_1311829/1274971694.py:3: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/tmp/ipykernel_1311829/1274971694.py:4: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


`-` 시각화2의 수정

In [13]:
df.rename({'지역/연도[6]':'지역'},axis=1)\
.set_index(['지역'])\
.applymap(lambda x: 0 if '-' in str(x) else float(x)).drop('전국')\
.applymap(lambda x: x*1000 if x<1000 else x)\
.stack().reset_index().rename({'level_1':'년도',0:'출생아수'},axis=1)\
.plot.line(x='년도',y='출생아수',color='지역',backend='plotly')

/tmp/ipykernel_1311829/1526382732.py:3: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/tmp/ipykernel_1311829/1526382732.py:4: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


`-` 시각화3의 수정

In [14]:
df.rename({'지역/연도[6]':'지역'},axis=1)\
.set_index(['지역'])\
.applymap(lambda x: 0 if '-' in str(x) else float(x)).drop('전국')\
.applymap(lambda x: x*1000 if x<1000 else x)\
.stack().reset_index().rename({'level_1':'년도',0:'출생아수'},axis=1)\
.plot.area(x='년도',y='출생아수',color='지역',backend='plotly')

/tmp/ipykernel_1311829/3339566067.py:3: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/tmp/ipykernel_1311829/3339566067.py:4: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.
